In [102]:
# Imports
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader,
)  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!

In [103]:
import numpy as np

In [104]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.fc1 = nn.Linear(16 * 7 * 7, 250) # replace this with siren.
        self.fc2 = nn.Linear(250, 250)
        self.fc3 = nn.Linear(250, num_classes) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x)) # replace this with siren.
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [105]:
device = torch.device('cuda')

In [106]:
# Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 3e-4 # karpathy's constant
batch_size = 64
num_epochs = 3

In [107]:
# Load Data
train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [108]:
# Initialize network
model_relu = CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_relu.parameters(), lr=learning_rate)

In [109]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model_relu(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████| 938/938 [00:15<00:00, 60.70it/s]


In [110]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model_relu)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_relu)*100:.2f}")

Accuracy on training set: 97.29
Accuracy on test set: 97.25


In [111]:
# Our pretrained nn for adversial attack is model, which is a relu network

In [112]:
def compute_gradient (func,inp, **kwargs):
    inp.requires_grad = True
    loss = func(inp, **kwargs)
    loss.backward()
    inp.requires_grad = False
    return inp.grad.data

In [113]:
def func(inp, model = None, target = None):
    out = model(inp)
    loss = torch.nn.functional.nll_loss(out,torch.LongTensor([target]).to(device))
    #print(f"Loss:  {loss.item()}")
    return loss

In [114]:
def attack (tensor, model, eps = 1e-3, n_iter = 5000):
    number = 5000
    new_tensor = tensor.unsqueeze(0).detach().clone()
    new_tensor = new_tensor.to(device = device)
    orig_prediction = model(new_tensor).argmax()
    #print(f"Original Prediction: {orig_prediction.item()}")

    for i in tqdm(range(n_iter)):
        model.zero_grad()

        grad = compute_gradient(func, new_tensor, model = model, target = orig_prediction.item())
        #new_tensor = new_tensor + eps * grad.sign()
        new_tensor = torch.clamp(new_tensor + eps * grad.sign(), -2, 2)
        new_prediction = model(new_tensor).argmax()

        if new_prediction != orig_prediction:
            #print(f"New Prediction: {new_prediction.item()}")
            number = i
            break
    return number
    

In [115]:
num1 = attack(train_dataset[0][0], model_relu)
num1

  1%|          | 46/5000 [00:00<00:14, 339.33it/s]


46

In [116]:
# now its time for siren.
class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.
    
    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the 
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a 
    # hyperparameter.
    
    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of 
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)
    
    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30,is_linear = False):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.is_linear = is_linear
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        self.init_weights()
    
    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 
                                             1 / self.in_features)      
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0, 
                                             np.sqrt(6 / self.in_features) / self.omega_0)
        
    def forward(self, input):
        if self.is_linear:
          return (self.omega_0 * self.linear(input))
        return torch.sin(self.omega_0 * self.linear(input))
    
    def forward_with_intermediate(self, input): 
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate
    
    
class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, outermost_linear=False, 
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()
        
        self.net = []
        self.net.append(SineLayer(in_features, hidden_features, 
                                  is_first=True, omega_0=first_omega_0))

        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features, 
                                      is_first=False, omega_0=hidden_omega_0))

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            
            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0, 
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)
                
            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features, 
                                      is_first=False, omega_0=hidden_omega_0))
        
        self.net = nn.Sequential(*self.net)
    
    def forward(self, coords):
        coords = coords.clone().detach().requires_grad_(True) # allows to take derivative w.r.t. input
        output = self.net(coords)
        return output, coords        

    def forward_with_activations(self, coords, retain_grad=False):
        '''Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!'''
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)
        activations['input'] = x
        for i, layer in enumerate(self.net):
            if isinstance(layer, SineLayer):
                x, intermed = layer.forward_with_intermediate(x)
                
                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()
                    
                activations['_'.join((str(layer.__class__), "%d" % activation_count))] = intermed
                activation_count += 1
            else: 
                x = layer(x)
                
                if retain_grad:
                    x.retain_grad()
                    
            activations['_'.join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations

In [117]:
# Simple CNN
class siren_CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(siren_CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        #self.fc1 = nn.Linear(16 * 7 * 7, 250) # replace this with siren.
        self.fc1 = SineLayer(16 * 7 * 7, 250, is_first=True, omega_0=30)
        self.fc2 = SineLayer(250, 250, is_first=False, omega_0=30)
        #self.fc3 = SineLayer(250, 250, is_first=False, omega_0=30)
        #self.fc4 = SineLayer(250, 250, is_first=False, omega_0=30)
        self.fc3 =  SineLayer(250, num_classes, is_first=False, omega_0=30)
        #self.fc2 = SineLayer(250, num_classes, is_first=False, omega_0=30, is_linear=True)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        #x = F.relu(self.fc1(x)) # replace this with siren.
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        #x = self.fc4(x)
        #x = self.fc5(x)
        return x


In [118]:
# Initialize network
model_siren = siren_CNN(in_channels=in_channels, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_siren.parameters(), lr=learning_rate)

In [119]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model_siren(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

100%|██████████| 938/938 [00:16<00:00, 57.81it/s]


In [120]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model_siren)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model_siren)*100:.2f}")

Accuracy on training set: 98.48
Accuracy on test set: 98.23


In [121]:
model_siren(train_dataset[1][0].unsqueeze(0).to(device)).argmax()

tensor(0, device='cuda:0')

In [122]:
model_siren(train_dataset[1][0].unsqueeze(0).to(device)).argmax()

tensor(0, device='cuda:0')

In [123]:
train_dataset[1][1]

0

In [124]:
num1 = attack(train_dataset[0][0], model_siren)
num1

  0%|          | 10/5000 [00:00<00:17, 284.11it/s]


10

In [125]:
def compute_avg_n_iter (model):
    sum = 0
    for i in tqdm(range(1000)):
        sum += attack(train_dataset[i][0], model)
    return sum/1000

In [126]:
def new_compute_avg_n_iter (model):
    sum = 0
    counter = 0
    i = 0
    while(counter<1000):
        siren_prediction = model_siren(train_dataset[i][0].unsqueeze(0).to(device)).argmax()
        relu_prediction = model_relu(train_dataset[i][0].unsqueeze(0).to(device)).argmax()
        actual = train_dataset[i][1]
        if (siren_prediction == actual and relu_prediction == actual):
            sum += attack(train_dataset[i][0], model)
            counter += 1
        i += 1
    return sum/1000

In [127]:
def new_compute_avg_n_iter_test (model):
    sum = 0
    counter = 0
    i = 0
    while(counter<1000):
        siren_prediction = model_siren(test_dataset[i][0].unsqueeze(0).to(device)).argmax()
        relu_prediction = model_relu(test_dataset[i][0].unsqueeze(0).to(device)).argmax()
        actual = test_dataset[i][1]
        print(counter)
        if (siren_prediction == actual and relu_prediction == actual):
            sum += attack(test_dataset[i][0], model)
            counter += 1
        i += 1
    return sum/1000

In [128]:
print(f"avg n_iter for model_relu = {new_compute_avg_n_iter_test(model_relu)}")

0


  2%|▏         | 117/5000 [00:00<00:13, 358.40it/s]


1


  3%|▎         | 126/5000 [00:00<00:15, 308.07it/s]


2


...

998


  3%|▎         | 129/5000 [00:00<00:17, 278.39it/s]


999


  1%|▏         | 71/5000 [00:00<00:16, 295.86it/s]

avg n_iter for model_relu = 91.738


In [129]:
print(f"avg n_iter for model_siren = {new_compute_avg_n_iter_test(model_siren)}")

0


  3%|▎         | 131/5000 [00:00<00:25, 194.31it/s]


1


  1%|          | 35/5000 [00:00<00:22, 222.11it/s]


2


...

998


  5%|▌         | 255/5000 [00:00<00:14, 324.54it/s]


999


 10%|█         | 513/5000 [00:01<00:13, 328.21it/s]

avg n_iter for model_siren = 1163.835


In [130]:
check_counter

0